In [8]:
import pandas as pd
import pyarrow.parquet as pq

import pyspark
from pyspark.sql import SparkSession, functions as F
from pyspark.sql import types

In [2]:
spark = SparkSession.builder.master("local[*]").appName('test').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/11 07:32:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-01.parquet

--2022-10-10 08:33:15--  https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 13.224.194.91, 13.224.194.4, 13.224.194.107, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|13.224.194.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 308924937 (295M) [application/x-www-form-urlencoded]
Saving to: ‘fhvhv_tripdata_2021-01.parquet’

fhvhv_tripdata_2021 100%[===================>] 294.61M  9.54MB/s    in 34s     

2022-10-10 08:33:51 (8.60 MB/s) - ‘fhvhv_tripdata_2021-01.parquet’ saved [308924937/308924937]



In [49]:
!wc -l fhvhv_tripdata_2021-01.parquet

1006794 fhvhv_tripdata_2021-01.parquet


In [3]:
df = spark.read.parquet("fhvhv_tripdata_2021-01.parquet")

In [4]:
df.head()

Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', originating_base_num='B02682', request_datetime=datetime.datetime(2021, 1, 1, 0, 28, 9), on_scene_datetime=datetime.datetime(2021, 1, 1, 0, 31, 42), pickup_datetime=datetime.datetime(2021, 1, 1, 0, 33, 44), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 49, 7), PULocationID=230, DOLocationID=166, trip_miles=5.26, trip_time=923, base_passenger_fare=22.28, tolls=0.0, bcf=0.67, sales_tax=1.98, congestion_surcharge=2.75, airport_fee=None, tips=0.0, driver_pay=14.99, shared_request_flag='N', shared_match_flag='N', access_a_ride_flag=' ', wav_request_flag='N', wav_match_flag='N')

In [5]:
df.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('originating_base_num', StringType(), True), StructField('request_datetime', TimestampType(), True), StructField('on_scene_datetime', TimestampType(), True), StructField('pickup_datetime', TimestampType(), True), StructField('dropoff_datetime', TimestampType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('trip_miles', DoubleType(), True), StructField('trip_time', LongType(), True), StructField('base_passenger_fare', DoubleType(), True), StructField('tolls', DoubleType(), True), StructField('bcf', DoubleType(), True), StructField('sales_tax', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('airport_fee', DoubleType(), True), StructField('tips', DoubleType(), True), StructField('driver_pay', DoubleType(), True), StructField('shared_request_flag',

In [53]:
!head -n 1001 fhvhv_tripdata_2021-01.parquet > head.parquet

In [54]:
!ls

03_test.ipynb  04_test.ipynb  fhvhv_tripdata_2021-01.parquet  head.parquet


In [55]:
!wc -l head.parquet

1001 head.parquet


In [32]:
df_pandas = pd.read_parquet("fhvhv_tripdata_2021-01.parquet")

In [33]:
# df_pandas.dtypes

In [34]:
df_pandas[["hvfhs_license_num","dispatching_base_num","pickup_datetime",
           "dropoff_datetime","PULocationID","DOLocationID"]].to_csv("fhvhv_tripdata_2021-01.csv", index=False)

In [35]:
!ls

03_test.ipynb  fhvhv_tripdata_2021-01.csv      head.parquet
04_test.ipynb  fhvhv_tripdata_2021-01.parquet


In [36]:
# df_pandas = pd.read_csv("fhvhv_tripdata_2021-01.csv")

In [38]:
# spark.createDataFrame(df_pandas).schema

In [39]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True)
])

In [40]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-01.csv')

In [41]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-----------------+--------------------+-------------------+-------------------+------------+------------+
|           HV0003|              B02682|2021-01-01 00:33:44|2021-01-01 00:49:07|         230|         166|
|           HV0003|              B02682|2021-01-01 00:55:19|2021-01-01 01:18:21|         152|         167|
|           HV0003|              B02764|2021-01-01 00:23:56|2021-01-01 00:38:05|         233|         142|
|           HV0003|              B02764|2021-01-01 00:42:51|2021-01-01 00:45:50|         142|         143|
|           HV0003|              B02764|2021-01-01 00:48:14|2021-01-01 01:08:42|         143|          78|
|           HV0005|              B02510|2021-01-01 00:06:59|2021-01-01 00:43:01|          88|          42|
|           HV0005|              B025

In [43]:
df.head(5)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 33, 44), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 49, 7), PULocationID=230, DOLocationID=166),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 55, 19), dropoff_datetime=datetime.datetime(2021, 1, 1, 1, 18, 21), PULocationID=152, DOLocationID=167),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 23, 56), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 38, 5), PULocationID=233, DOLocationID=142),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 42, 51), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 45, 50), PULocationID=142, DOLocationID=143),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 48, 14), dropoff_

In [44]:
df.repartition(24)

DataFrame[hvfhs_license_num: string, dispatching_base_num: string, pickup_datetime: timestamp, dropoff_datetime: timestamp, PULocationID: int, DOLocationID: int]

In [45]:
df = df.repartition(24)

In [46]:
df.write.parquet('fhvhv/2021/01/')

In [3]:
!ls

03_test.ipynb  fhvhv			   fhvhv_tripdata_2021-01.parquet
04_test.ipynb  fhvhv_tripdata_2021-01.csv  head.parquet


In [4]:
df = spark.read.parquet('fhvhv/2021/01/')

In [5]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)



In [7]:
df.select("pickup_datetime","dropoff_datetime", "PULocationID", "DOLocationID")\
            .filter(df.hvfhs_license_num=="HV0003")\
            .show()

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2021-01-01 18:18:27|2021-01-01 18:30:13|         255|          97|
|2021-01-06 23:03:39|2021-01-06 23:10:45|         230|          50|
|2021-01-06 09:32:30|2021-01-06 09:40:02|         215|         130|
|2021-01-01 10:09:08|2021-01-01 10:21:59|          86|         265|
|2021-01-03 22:45:14|2021-01-03 23:40:43|         132|         265|
|2021-01-03 18:42:25|2021-01-03 19:13:53|         223|          14|
|2021-01-05 21:56:35|2021-01-05 22:18:37|         125|         127|
|2021-01-03 18:31:14|2021-01-03 18:54:33|          61|         198|
|2021-01-03 23:27:38|2021-01-03 23:36:47|         256|          17|
|2021-01-06 21:14:59|2021-01-06 21:26:32|         210|          11|
|2021-01-06 10:22:18|2021-01-06 10:34:56|         248|         159|
|2021-01-02 12:27:47|2021-01-02 12:37:23|       